In [1]:
from imports import *
from arch import *
model = return_token_model('distilbert-base-uncased', len(labels), preferred_cuda_test);
model.load_state_dict(torch.load(CHECKPOINT_PATH + ROOT_NAME + '2_12750.pt')["model_state_dict"]);
model.eval();

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [2]:
from shared_train_and_test_functions import tokenization
# ONLY WORKS FOR SINGLE POST TESTING
post = ["I am positive that Covid vaccines are bad."]

import nltk
words = nltk.word_tokenize(post[0])
test_encodings, tokens = tokenization(post, True)
print(test_encodings, tokens)
print(len(post[0]))

{'input_ids': [[101, 1045, 2572, 3893, 2008, 2522, 17258, 28896, 2024, 2919, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'offset_mapping': [[(0, 0), (0, 1), (2, 4), (5, 13), (14, 18), (19, 21), (21, 24), (25, 33), (34, 37), (38, 41), (41, 42), (0, 0)]]} ['[CLS]', 'i', 'am', 'positive', 'that', 'co', '##vid', 'vaccines', 'are', 'bad', '.', '[SEP]']
42


In [3]:
input_ids = torch.tensor(test_encodings['input_ids']).to(preferred_cuda_test)
attention_mask = torch.tensor(test_encodings['attention_mask']).to(preferred_cuda_test)
outputs = model(input_ids, attention_mask=attention_mask) # try this: labels = torch.tensor([0]*len(input_ids)).to(preferred_cuda_test))
predictions = outputs[0].detach().cpu().numpy()
print(predictions)
single_prediction_post = predictions[0]
# arg max on each row
predictions_argmax = np.argmax(single_prediction_post, axis=1)
print(predictions_argmax)

[[[ 4.288851   -1.8353655  -1.975081  ]
  [ 4.9777536  -1.0429281  -3.4052281 ]
  [ 5.441175   -1.560985   -3.2416413 ]
  [ 3.4962287  -1.9294155  -1.6872611 ]
  [ 1.877359   -2.2214005   0.28020906]
  [ 0.98067564 -3.0352888   2.3563764 ]
  [-1.0166444  -2.932358    3.4987018 ]
  [-0.7718418  -2.4467847   2.953644  ]
  [ 4.081329   -1.4025064  -1.7969477 ]
  [-0.62212163 -2.331483    2.8353157 ]
  [ 0.44208255 -2.1755955   1.9302607 ]
  [ 4.7132397  -2.0740929  -2.0252438 ]]]
[0 0 0 0 0 2 2 2 0 2 2 0]


In [5]:
for i,index in enumerate(predictions_argmax):
    if tokens[i] in ["i", "co", "##vid", "vaccines"]:
        label = True
    else:
        label = False
    print(reverse_labels[index] + " ---> " + tokens[i] + " ---> " + "entity?" + " ---> " + label)

NameError: name 'token' is not defined